# Analyse av Trondheim Forecast

#### Import av nødvendige moduler og klasser:

In [1]:
import sys
import os
import pandas as pd
import numpy as np

sys.path.append(os.path.abspath("../src"))


from data_analysis import DataAnalyse
from data_reader import DataLeser

#### Last inn renset data

In [2]:
dataleser= DataLeser(data_dir="../data")
df_renset = dataleser.les_csv("trondheim_forecast_cleaned.csv")

#### Initialiser analyseobjekt:

In [3]:
analyse = DataAnalyse(df_renset)

#### Statistisk analyse:
- Bruker pandas sine .mean(), .median() og .std() til å beregne de ønskede statistiske målene. 

In [4]:
stat_summary = analyse.beskriv_data()
print(stat_summary)

               Gjennomsnitt   Median  Standardavvik
Temperatur        12.295349    12.15       3.094128
Fuktighet         76.436047    78.55      13.667358
Trykk           1003.611628  1002.45       4.215334
Vindhastighet      2.633721     2.40       1.265247


Hvorfor bruke gjennomsnitt, median og standardavvik? 
- Gjennomsnitt gir et mål på sentraltrender, men kan påvirkes av ekstremverdier som ofte oppstår i naturen.
- Median virker bra mot outliers og gir et stabilt bilde ved skjevfordelte data.
- Standardavvik sier noe om hvor mye verdiene i datasettet varierer rundt snittet.

#### Korrelasjon mellom Temperatur og Fuktighet:
- Viser sammenheng mellom to verdier i datasettet ved en korrelasjonskoeffisient (Pearson) som sier noe om hvor sterkt to variabler henger sammen. 
    - Positiv: temperatur stiger -> øker fuktighet
    - Negativ: temperatur stiger -> synker fuktighet
    - Nær null: liten eller ingen lineær sammenheng.

In [5]:
kol1="Temperatur"
kol2="Fuktighet"
try:
    korrelasjon = analyse.korrelasjon(kol1, kol2)
    print(f"Korrelasjon mellom {kol1} og {kol2}: {korrelasjon:.3f}")
except ValueError as e:
    print(e)

Korrelasjon mellom Temperatur og Fuktighet: -0.850


Vi ser at når temperaturen øker så har fuktigheten synket. Dette gir menning. 
Varme luftmasser kan holde på mer vanndamp som impliserer at relativ fuktighet synker selv om mengden damp er lik.
- Dermed; på varme, tørre dager ser vi ofte en lav relativ fuktighet. Mens på kalde netter øker relativ fuktighet.

#### Indetifiser Outliers:
- Outliers kan oppstå på grunn av feil i måling eller ekstreme hendelser.
- I analysen av vær så vil outliers ofte oppstå grunnet endringer i vær. vi kan identifisere dem og senere inkludere dem i visualiseringen og prediktive modelleringer for å se hvordan de påvirker de ikke uteliggende verdiene.

In [6]:
outliers = analyse.identifiser_outliers(threshold=2.5)
print(f"Antall rader der outliers identifiseres: {len(outliers)}")

Antall rader der outliers identifiseres: 8


- Metoden finner outliers ved å beregne en score:
$$
\text{score} = \frac{{observasjonsverdi} - \text{gjennomsnitt}}{\text{standardavvik}}
$$
- En høy verdi for score impliserer at det er en outlier.
- Metoden bestemmer terskelverdien på 2.5 som betyr at en verdi ligger 2.5 standardavvik unna gjennomsnittet.

- Vi ser at analysen gir en lav mengde outliers. Vi vet at dette er korrekt siden dersom vi minker denne verdier så får vi mange flere uteliggere. Dette er grunnet det av vi har et lite datasett som har god datakvalitet. Originalt hadde vi 3.0 som terskel, men endret dette siden vi fikk ingen outliers.

#### Trendanalyse for temperatur:
- Trendanalyse med glidende gjenomsnitt

Ved å bruke rullerende gjennomsnitt over for eksempel 7 dager, glatter vi ut tilfeldige svingiger. Dette gir bedre innsikt for:
- Klimatendens over uker.
- Ujevnheter pga. værskifte

In [7]:
temp_trend = analyse.kolonne_trend("Temperatur", vindu=7)
print("Glidende 7-dagers snitt for temperatur:")
print(temp_trend.head(10))

Glidende 7-dagers snitt for temperatur:
0    11.700000
1    10.900000
2    10.266667
3     9.900000
4     9.600000
5     9.300000
6     9.071429
7     8.428571
8     7.928571
9     7.542857
Name: Temperatur, dtype: float64



- Verdiene vi får ut er gjennomsnittet over en 7 dagers periode. Denne viser en gradvis nedkjøling over perioden. De første verdiene er høyere og basert på få datapunkter, men etter at flere mplinger inkluderes, stabiliseres trendlinjen seg og gir et mer tydelig bilde.

- Et mulig feilestimat her er sorteringen av tid før rolling() brukes. Dersom senere datoer ligger først, kan dette gi feilretning på trenden.

- Disse verdiene er nyttige i neste steg som er Visualiseringen. 